In [2]:
import os,codecs
import pandas as pd
import nltk
from nltk import*
import matplotlib.pyplot as plt
import string
import numpy as np
import seaborn as sns


In [3]:
df = pd.read_csv('/Users/ganiyuopeyemi/desktop/NLP/XED-master/AnnotatedData/en-annotated.tsv',sep='\t',header = None, names = ["sent", "label_1"], error_bad_lines=False)
df.head()

,sent,label_1
0,", ...",1
1,!,"1, 4, 7"
2,... And I don't think we need to discuss the T...,"8, 1"
3,* So get up out of your bed,1
4,A confession that you hired [PERSON] ... and a...,"1, 6"


In [4]:
df.shape

(17528, 2)

In [5]:
df[["label_1", "label_2"]] = df['label_1'].str.split(",", 1, expand=True)
df[["label_2", "label_3"]] = df['label_2'].str.split(",", 1, expand=True)

In [6]:
df.head()

,sent,label_1,label_2,label_3
0,", ...",1,None,None
1,!,1,4,7
2,... And I don't think we need to discuss the T...,8,1,None
3,* So get up out of your bed,1,None,None
4,A confession that you hired [PERSON] ... and a...,1,6,None


In [7]:
df["label_1"] = pd.to_numeric(df["label_1"])
df["label_2_fill"] = np.where(df.label_2.isnull(), df.label_1, df.label_2)
df["label_2_fill"] = pd.to_numeric(df["label_2_fill"])

In [8]:
lst_stopwords = nltk.corpus.stopwords.words("english")

In [9]:
#stemmer = SnowballStemmer("english")

In [10]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    
               
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    table = str.maketrans('', '', string.punctuation)
    lst_text = [w.translate(table) for w in lst_text]
    #lst_text = [stemmer.stem(w.lower()) for w in lst_text]
    ## back to string from list
    text = " ".join(lst_text)
    return text


In [11]:
df["text_clean"] = df["sent"].apply(lambda x: 
          utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, 
          lst_stopwords= lst_stopwords))

In [12]:
df['tokenized_sents'] = df.apply(lambda row: nltk.word_tokenize(row["text_clean"]), axis=1)
df.head()

,sent,label_1,label_2,label_3,label_2_fill,text_clean,tokenized_sents
0,", ...",1,None,None,1,,[]
1,!,1,4,7,4,,[]
2,... And I don't think we need to discuss the T...,8,1,None,1,and i dont think we need to discuss the trinit...,"[and, i, dont, think, we, need, to, discuss, t..."
3,* So get up out of your bed,1,None,None,1,so get up out of your bed,"[so, get, up, out, of, your, bed]"
4,A confession that you hired [PERSON] ... and a...,1,6,None,6,a confession that you hired person and are res...,"[a, confession, that, you, hired, person, and,..."


In [13]:
df = df[["sent","text_clean", "label_1", "label_2_fill","tokenized_sents"]]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17528 entries, 0 to 17527
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sent             17528 non-null  object
 1   text_clean       17528 non-null  object
 2   label_1          17528 non-null  int64 
 3   label_2_fill     17528 non-null  int64 
 4   tokenized_sents  17528 non-null  object
dtypes: int64(2), object(3)
memory usage: 684.8+ KB


In [15]:
h = df[df["text_clean"] == ""]

In [16]:
h.shape

(3, 5)

In [17]:
h

,sent,text_clean,label_1,label_2_fill,tokenized_sents
0,", ...",,1,1,[]
1,!,,1,4,[]
3160,% ? !,,7,7,[]


In [18]:
h = df[df["text_clean"] == ""].index
df.drop(h, inplace=True)

In [19]:
df = df.reset_index(drop=True)

In [20]:
df.shape

(17525, 5)

In [21]:
df.head()

,sent,text_clean,label_1,label_2_fill,tokenized_sents
0,... And I don't think we need to discuss the T...,and i dont think we need to discuss the trinit...,8,1,"[and, i, dont, think, we, need, to, discuss, t..."
1,* So get up out of your bed,so get up out of your bed,1,1,"[so, get, up, out, of, your, bed]"
2,A confession that you hired [PERSON] ... and a...,a confession that you hired person and are res...,1,6,"[a, confession, that, you, hired, person, and,..."
3,A dead man has one half - hour to raise his ro...,a dead man has one half hour to raise his roll...,1,1,"[a, dead, man, has, one, half, hour, to, raise..."
4,A guy that's talking about he's gonna solve al...,a guy thats talking about hes gonna solve all ...,1,1,"[a, guy, thats, talking, about, hes, gon, na, ..."


In [22]:
file = "NRC_emotion_lexicon_list.txt"
emolex_df = pd.read_csv(file,  names=["word", "emotion", "association"], skiprows=45, sep='\t')
emolex_df.head(10)

,word,emotion,association
0,abandonment,negative,1
1,abandonment,positive,0
2,abandonment,sadness,1
3,abandonment,surprise,1
4,abandonment,trust,0
5,abate,anger,0
6,abate,anticipation,0
7,abate,disgust,0
8,abate,fear,0
9,abate,joy,0


In [23]:
emolex_df.shape

(141775, 3)

In [24]:
emolex_words = emolex_df.pivot(index='word', columns='emotion', values='association').reset_index()

In [25]:
emotions = emolex_words.columns.drop('word')
emo_df = pd.DataFrame(0, index=df.index, columns=emotions)

In [26]:
emotions

Index(['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative',
       'positive', 'sadness', 'surprise', 'trust'],
      dtype='object', name='emotion')

In [27]:
emolex_words.head()

emotion,word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,abandonment,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,1.0,0.0
2,abate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,abatement,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,abba,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [28]:
word_list = []
for sent in df.tokenized_sents:
    for word in sent:
        word_list.append(word)

In [29]:
len(word_list)

130534

In [30]:
len(set(word_list))

8586

In [31]:
emo_words = []
for word in emolex_words.word:
    emo_words.append(word)
print(len(emo_words))

14178


In [32]:
match = []
for word in word_list:
    if word in emo_words:
        match.append(word)

In [33]:
print(len(match))
print(len(set(match)))

28707
4117


In [36]:
(len(set(match))/len(set(word_list)))*100

47.95015140927091

In [43]:
from nrclex import NRCLex

In [44]:
my_list = []
for row in df.text_clean:
    my_list.append(row)

In [45]:
len(my_list)

17525

In [46]:
text = df.text_clean[11]
text

'a lost decade but it was worth it to make your father pay for my loss'

In [47]:
text_object =  NRCLex(text)
d = text_object.raw_emotion_scores
print(d)

{'negative': 2, 'sadness': 2, 'positive': 2, 'trust': 2, 'anticipation': 1, 'joy': 1, 'anger': 1, 'fear': 1}


In [48]:
data = pd.DataFrame()
dicts = []
neg = []
pos = []
fear = []
anger = []
disg = []
joy = []
anticipation = []
trust = []
surp = []
sad = []
for sent in my_list:
    text_object =  NRCLex(sent)
    d = text_object.raw_emotion_scores
    for j in emotions:
        if j not in d.keys():
            d[j] =0
    dicts.append(d)
    
for dic in dicts:
    neg.append(dic["negative"])
    pos.append(dic["positive"])
    fear.append(dic["fear"])
    anger.append(dic["anger"])
    disg.append(dic["disgust"])
    joy.append(dic["joy"])
    anticipation.append(dic["anticipation"])
    trust.append(dic["trust"])
    surp.append(dic["surprise"])
    sad.append(dic["sadness"])
   


In [49]:
print(len(pos))
print(len(neg))
print(len(fear))
print(len(anger))
print(len(disg))
print(len(joy))
print(len(anticipation))
print(len(trust))
print(len(surp))
print(len(sad))

17525
17525
17525
17525
17525
17525
17525
17525
17525
17525


In [50]:
data["cleaned"] = my_list
data["pos"] = pos
data["neg"] = neg
data["fear"] = fear
data["anger"] = anger
data["disg"] = disg
data["joy"] = joy
data["anticipation"] = anticipation
data["trust"] = trust
data["surp"] = surp
data["sad"] = sad

data.head()

,cleaned,pos,neg,fear,anger,disg,joy,anticipation,trust,surp,sad
0,and i dont think we need to discuss the trinit...,0,0,0,0,0,0,0,0,0,0
1,so get up out of your bed,0,0,0,0,0,0,0,0,0,0
2,a confession that you hired person and are res...,1,2,2,1,1,0,1,1,2,2
3,a dead man has one half hour to raise his roll...,0,0,0,0,0,0,0,0,0,0
4,a guy thats talking about hes gonna solve all ...,0,0,0,0,0,0,0,0,0,0


In [51]:
data["sum"] = data.sum(axis=1)

In [52]:
data["sum"].value_counts()

0     7667
1     2149
2     1738
3     1543
5     1302
4     1150
6      666
7      377
8      282
9      212
10     138
11      93
12      71
13      48
15      27
16      20
14      18
17       8
19       5
18       5
22       4
20       1
21       1
Name: sum, dtype: int64

In [53]:
data["label_1"] = df.label_1
data["label_2_fill"] = df.label_2_fill

In [54]:
data = data.drop(data[data["sum"] == 0].index)

In [55]:
data = data.reset_index(drop=True)
data.head()

,cleaned,pos,neg,fear,anger,disg,joy,anticipation,trust,surp,sad,sum,label_1,label_2_fill
0,a confession that you hired person and are res...,1,2,2,1,1,0,1,1,2,2,13,1,6
1,a guy who was a pain in the neck even before i...,0,1,1,0,0,0,0,0,0,1,3,1,1
2,a hundred of these are produced every day and ...,1,1,1,1,1,0,1,0,0,1,7,1,3
3,a little restraint,1,0,0,0,0,0,0,0,0,0,1,1,1
4,a lost decade but it was worth it to make your...,2,2,1,1,0,1,1,2,0,2,12,1,1


In [56]:
data.shape

(9858, 14)

In [57]:
data["pos_target"] = data["joy"] + data["anticipation"] + data["trust"]
data["neg_target"] = data["fear"] + data["anger"] + data["disg"] + data["surp"] + data["sad"]

In [58]:
data.head()

,cleaned,pos,neg,fear,anger,disg,joy,anticipation,trust,surp,sad,sum,label_1,label_2_fill,pos_target,neg_target
0,a confession that you hired person and are res...,1,2,2,1,1,0,1,1,2,2,13,1,6,2,8
1,a guy who was a pain in the neck even before i...,0,1,1,0,0,0,0,0,0,1,3,1,1,0,2
2,a hundred of these are produced every day and ...,1,1,1,1,1,0,1,0,0,1,7,1,3,1,4
3,a little restraint,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0
4,a lost decade but it was worth it to make your...,2,2,1,1,0,1,1,2,0,2,12,1,1,4,4


In [59]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.linear_model import LogisticRegression


# Polarity based on the sum of positive emotions vs the sum of the negative emotions (XED vs EMOLEX)

# label_1 vs sums

In [60]:
data_diff = data.copy()
data_diff.drop(data_diff.loc[data_diff["pos_target"]==data_diff["neg_target"]].index, inplace=True)


In [61]:
data_diff = data_diff.reset_index(drop=True)
data_diff.shape

(8151, 16)

In [62]:
data_diff["target"] = data_diff[["pos_target", "neg_target"]].idxmax(axis=1)

In [63]:
data_diff = data_diff[["cleaned", "label_1", "label_2_fill", "target"]]

In [64]:
data_diff["label_polarity"] = data_diff["label_1"].map({1:0, 2:1, 3:0, 4:0, 5:1, 6:0, 7:0, 8:1})
data_diff["label_polarity_2"] = data_diff["label_2_fill"].map({1:0, 2:1, 3:0, 4:0, 5:1, 6:0, 7:0, 8:1})
data_diff["target_polarity"] = data_diff["target"].map({'neg_target':0, 'pos_target':1})

In [139]:
X = data_diff["cleaned"]
y_target = data_diff["target_polarity"]
y_label = data_diff["label_polarity"]

In [140]:
data_diff.label_polarity.value_counts()

0    4850
1    3301
Name: label_polarity, dtype: int64

In [141]:
data_diff.target_polarity.value_counts()

1    4152
0    3999
Name: target_polarity, dtype: int64

In [142]:
# Naïve Bayes:
text_clf_nb = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB()),
])

# Linear SVC:
text_clf_lsvc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# LogisticRegression 
text_clf_lr = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LogisticRegression()),
])

In [143]:
X_train, X_test, y_train_t, y_test_t, y_train_l, y_test_l = train_test_split(X, y_target, y_label, test_size=0.30, random_state=42)

In [144]:
print("Report of MultinomialNB EmoLex Polarity (sums vs label_1)")
text_clf_nb.fit(X_train, y_train_t)
predictions = text_clf_nb.predict(X_test)
print("Confusion Matrix \n",metrics.confusion_matrix(y_test_l,predictions))
print(metrics.classification_report(y_test_l,predictions))
print("Accuracy Score: ", metrics.accuracy_score(y_test_l,predictions))

Report of MultinomialNB EmoLex Polarity (sums vs label_1)
Confusion Matrix 
 [[818 633]
 [281 714]]
              precision    recall  f1-score   support

           0       0.74      0.56      0.64      1451
           1       0.53      0.72      0.61       995

    accuracy                           0.63      2446
   macro avg       0.64      0.64      0.63      2446
weighted avg       0.66      0.63      0.63      2446

Accuracy Score:  0.6263286999182338


In [145]:
print("Report of LinearSVC EmoLex Polarity (sums vs label_1)")
text_clf_lsvc.fit(X_train, y_train_t)
predictions = text_clf_lsvc.predict(X_test)
print("Confusion Matrix \n", metrics.confusion_matrix(y_test_l,predictions))
print(metrics.classification_report(y_test_l,predictions))
print("Accuracy Score: ", metrics.accuracy_score(y_test_l,predictions))

Report of LinearSVC EmoLex Polarity (sums vs label_1)
Confusion Matrix 
 [[872 579]
 [309 686]]
              precision    recall  f1-score   support

           0       0.74      0.60      0.66      1451
           1       0.54      0.69      0.61       995

    accuracy                           0.64      2446
   macro avg       0.64      0.65      0.63      2446
weighted avg       0.66      0.64      0.64      2446

Accuracy Score:  0.6369582992641046


In [146]:
print("Report of LogisticRegression EmoLex Polarity (sums vs label_1)")
text_clf_lr.fit(X_train, y_train_t)
predictions = text_clf_lr.predict(X_test)
print("Confusion Matrix \n",metrics.confusion_matrix(y_test_l,predictions))
print(metrics.classification_report(y_test_l,predictions))
print("Accuracy Score", metrics.accuracy_score(y_test_l,predictions))

Report of LogisticRegression EmoLex Polarity (sums vs label_1)
Confusion Matrix 
 [[855 596]
 [317 678]]
              precision    recall  f1-score   support

           0       0.73      0.59      0.65      1451
           1       0.53      0.68      0.60       995

    accuracy                           0.63      2446
   macro avg       0.63      0.64      0.62      2446
weighted avg       0.65      0.63      0.63      2446

Accuracy Score 0.6267375306623058


# label_2 vs sums

In [147]:
y_label = data_diff["label_polarity_2"]
X_train, X_test, y_train_t, y_test_t, y_train_l, y_test_l = train_test_split(X, y_target, y_label, test_size=0.30, random_state=42)

In [148]:
data_diff.label_polarity_2.value_counts()

0    5005
1    3146
Name: label_polarity_2, dtype: int64

In [149]:
print("Report of MultinomialNB EmoLex Polarity (sums vs label_2)")
text_clf_nb.fit(X_train, y_train_t)
predictions = text_clf_nb.predict(X_test)
print("Confusion Matrix \n",metrics.confusion_matrix(y_test_l,predictions))
print(metrics.classification_report(y_test_l,predictions))
print("Accuracy Score: ", metrics.accuracy_score(y_test_l,predictions))

Report of MultinomialNB EmoLex Polarity (sums vs label_2)
Confusion Matrix 
 [[832 662]
 [267 685]]
              precision    recall  f1-score   support

           0       0.76      0.56      0.64      1494
           1       0.51      0.72      0.60       952

    accuracy                           0.62      2446
   macro avg       0.63      0.64      0.62      2446
weighted avg       0.66      0.62      0.62      2446

Accuracy Score:  0.6201962387571546


In [150]:
print("Report of LinearSVC EmoLex Polarity (sums vs label_2)")
text_clf_lsvc.fit(X_train, y_train_t)
predictions = text_clf_lsvc.predict(X_test)
print("Confusion Matrix \n", metrics.confusion_matrix(y_test_l,predictions))
print(metrics.classification_report(y_test_l,predictions))
print("Accuracy Score: ", metrics.accuracy_score(y_test_l,predictions))

Report of LinearSVC EmoLex Polarity (sums vs label_2)
Confusion Matrix 
 [[884 610]
 [297 655]]
              precision    recall  f1-score   support

           0       0.75      0.59      0.66      1494
           1       0.52      0.69      0.59       952

    accuracy                           0.63      2446
   macro avg       0.63      0.64      0.63      2446
weighted avg       0.66      0.63      0.63      2446

Accuracy Score:  0.6291905151267375


In [151]:
print("Report of LogisticRegression EmoLex Polarity (sums vs label_2)")
text_clf_lr.fit(X_train, y_train_t)
predictions = text_clf_lr.predict(X_test)
print("Confusion Matrix \n",metrics.confusion_matrix(y_test_l,predictions))
print(metrics.classification_report(y_test_l,predictions))
print("Accuracy Score", metrics.accuracy_score(y_test_l,predictions))

Report of LogisticRegression EmoLex Polarity (sums vs label_2)
Confusion Matrix 
 [[869 625]
 [303 649]]
              precision    recall  f1-score   support

           0       0.74      0.58      0.65      1494
           1       0.51      0.68      0.58       952

    accuracy                           0.62      2446
   macro avg       0.63      0.63      0.62      2446
weighted avg       0.65      0.62      0.63      2446

Accuracy Score 0.6206050695012265


# Polarity based on emolex (neg, pos)

# label_1 vs emolex

In [152]:
data_diff_1 = data.copy()
data_diff_1.drop(data_diff.loc[data_diff_1["pos"]==data_diff_1["neg"]].index, inplace=True)
data_diff_1 = data_diff_1.reset_index(drop=True)
data_diff_1.shape

(8206, 16)

In [153]:
data_diff_1["target"] = data_diff_1[["pos", "neg"]].idxmax(axis=1)
data_diff_1 = data_diff_1[["cleaned", "label_1", "label_2_fill", "target"]]

In [154]:
data_diff_1["label_polarity"] = data_diff_1["label_1"].map({1:0, 2:1, 3:0, 4:0, 5:1, 6:0, 7:0, 8:1})
data_diff_1["label_polarity_2"] = data_diff_1["label_2_fill"].map({1:0, 2:1, 3:0, 4:0, 5:1, 6:0, 7:0, 8:1})
data_diff_1["target_polarity"] = data_diff_1["target"].map({'neg':0, 'pos':1})

In [155]:
X = data_diff_1["cleaned"]
y_target = data_diff_1["target_polarity"]
y_label = data_diff_1["label_polarity"]

In [156]:
X_train, X_test, y_train_t, y_test_t, y_train_l, y_test_l = train_test_split(X, y_target, y_label, test_size=0.30, random_state=42)

In [157]:
data_diff_1.label_polarity.value_counts()

0    4851
1    3355
Name: label_polarity, dtype: int64

In [158]:
data_diff_1.target_polarity.value_counts()

1    4461
0    3745
Name: target_polarity, dtype: int64

In [159]:
print("Report of MultinomialNB EmoLex Polarity 1")
text_clf_nb.fit(X_train, y_train_t)
predictions = text_clf_nb.predict(X_test)
print("Confusion Matrix \n",metrics.confusion_matrix(y_test_l,predictions))
print(metrics.classification_report(y_test_l,predictions))
print("Accuracy Score: ", metrics.accuracy_score(y_test_l,predictions))

Report of MultinomialNB EmoLex Polarity 1
Confusion Matrix 
 [[752 714]
 [204 792]]
              precision    recall  f1-score   support

           0       0.79      0.51      0.62      1466
           1       0.53      0.80      0.63       996

    accuracy                           0.63      2462
   macro avg       0.66      0.65      0.63      2462
weighted avg       0.68      0.63      0.63      2462

Accuracy Score:  0.6271324126726239


In [160]:
print("Report of LinearSVC EmoLex Polarity 1")
text_clf_lsvc.fit(X_train, y_train_t)
predictions = text_clf_lsvc.predict(X_test)
print("Confusion Matrix \n", metrics.confusion_matrix(y_test_l,predictions))
print(metrics.classification_report(y_test_l,predictions))
print("Accuracy Score: ", metrics.accuracy_score(y_test_l,predictions))

Report of LinearSVC EmoLex Polarity 1
Confusion Matrix 
 [[831 635]
 [238 758]]
              precision    recall  f1-score   support

           0       0.78      0.57      0.66      1466
           1       0.54      0.76      0.63       996

    accuracy                           0.65      2462
   macro avg       0.66      0.66      0.65      2462
weighted avg       0.68      0.65      0.65      2462

Accuracy Score:  0.6454102355808286


In [161]:
print("Report of LogisticRegression EmoLex Polarity 1")
text_clf_lr.fit(X_train, y_train_t)
predictions = text_clf_lr.predict(X_test)
print("Confusion Matrix \n",metrics.confusion_matrix(y_test_l,predictions))
print(metrics.classification_report(y_test_l,predictions))
print("Accuracy Score", metrics.accuracy_score(y_test_l,predictions))

Report of LogisticRegression EmoLex Polarity 1
Confusion Matrix 
 [[774 692]
 [233 763]]
              precision    recall  f1-score   support

           0       0.77      0.53      0.63      1466
           1       0.52      0.77      0.62       996

    accuracy                           0.62      2462
   macro avg       0.65      0.65      0.62      2462
weighted avg       0.67      0.62      0.62      2462

Accuracy Score 0.624289195775792


# label_2 vs emolex

In [162]:
y_label = data_diff_1["label_polarity_2"]

In [163]:
data_diff_1.label_polarity.value_counts()

0    4851
1    3355
Name: label_polarity, dtype: int64

In [164]:
X_train, X_test, y_train_t, y_test_t, y_train_l, y_test_l = train_test_split(X, y_target, y_label, test_size=0.30, random_state=42)

In [165]:
print("Report of MultinomialNB EmoLex Polarity 2")
text_clf_nb.fit(X_train, y_train_t)
predictions = text_clf_nb.predict(X_test)
print("Confusion Matrix \n",metrics.confusion_matrix(y_test_l,predictions))
print(metrics.classification_report(y_test_l,predictions))
print("Accuracy Score: ", metrics.accuracy_score(y_test_l,predictions))

Report of MultinomialNB EmoLex Polarity 2
Confusion Matrix 
 [[766 751]
 [190 755]]
              precision    recall  f1-score   support

           0       0.80      0.50      0.62      1517
           1       0.50      0.80      0.62       945

    accuracy                           0.62      2462
   macro avg       0.65      0.65      0.62      2462
weighted avg       0.69      0.62      0.62      2462

Accuracy Score:  0.6177904142973193


In [166]:
print("Report of LinearSVC EmoLex Polarity 2")
text_clf_lsvc.fit(X_train, y_train_t)
predictions = text_clf_lsvc.predict(X_test)
print("Confusion Matrix \n", metrics.confusion_matrix(y_test_l,predictions))
print(metrics.classification_report(y_test_l,predictions))
print("Accuracy Score: ", metrics.accuracy_score(y_test_l,predictions))

Report of LinearSVC EmoLex Polarity 2
Confusion Matrix 
 [[847 670]
 [222 723]]
              precision    recall  f1-score   support

           0       0.79      0.56      0.66      1517
           1       0.52      0.77      0.62       945

    accuracy                           0.64      2462
   macro avg       0.66      0.66      0.64      2462
weighted avg       0.69      0.64      0.64      2462

Accuracy Score:  0.6376929325751421


In [167]:
print("Report of LogisticRegression EmoLex Polarity 1")
text_clf_lr.fit(X_train, y_train_t)
predictions = text_clf_lr.predict(X_test)
print("Confusion Matrix \n",metrics.confusion_matrix(y_test_l,predictions))
print(metrics.classification_report(y_test_l,predictions))
print("Accuracy Score", metrics.accuracy_score(y_test_l,predictions))

Report of LogisticRegression EmoLex Polarity 1
Confusion Matrix 
 [[788 729]
 [219 726]]
              precision    recall  f1-score   support

           0       0.78      0.52      0.62      1517
           1       0.50      0.77      0.61       945

    accuracy                           0.61      2462
   macro avg       0.64      0.64      0.61      2462
weighted avg       0.67      0.61      0.62      2462

Accuracy Score 0.6149471974004874
